In [1]:
import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'
from object_detection.protos import pipeline_pb2
from google.protobuf import text_format
import tensorflow as tf

# Устанавливаем папки

In [2]:
# fish_name = input("Напишите с какой рыбой работаем:")
fish_name = 'осетр'
if fish_name.lower() == 'карп':
    LABEL_MAP_NAME = 'label_map_karp.pbtxt'
    name = 'karp'
elif fish_name.lower() == 'осетр':
    LABEL_MAP_NAME = 'label_map_osetr.pbtxt'
    name = 'osetr'
else:
    print('Пока нет фотографий для такой рыбы')

In [3]:
CUSTOM_MODEL_NAME = 'my_ssd_mobnet' 
PRETRAINED_MODEL_NAME = 'ssd_mobilenet_v2_fpnlite_320x320_coco17_tpu-8'
PRETRAINED_MODEL_URL = 'http://download.tensorflow.org/models/object_detection/tf2/20200711/ssd_mobilenet_v2_fpnlite_320x320_coco17_tpu-8.tar.gz'
TF_RECORD_SCRIPT_NAME = 'generate_tfrecord.py'

In [4]:
paths = {
    'WORKSPACE_PATH': os.path.join('Tensorflow', 'workspace'),
    'SCRIPTS_PATH': os.path.join('Tensorflow','scripts'),
    'APIMODEL_PATH': os.path.join('Tensorflow','models'),
    'ANNOTATION_PATH': os.path.join('Tensorflow', 'workspace','annotations'),
    'IMAGE_PATH': os.path.join('Tensorflow', 'workspace','data'),
    'MODEL_PATH': os.path.join('Tensorflow', 'workspace','models'),
    'PRETRAINED_MODEL_PATH': os.path.join('Tensorflow', 'workspace','pre-trained-models'),
    'CHECKPOINT_PATH': os.path.join('Tensorflow', 'workspace','models',CUSTOM_MODEL_NAME, name + '_checkpoints'), 
    'OUTPUT_PATH': os.path.join('Tensorflow', 'workspace','models',CUSTOM_MODEL_NAME, 'export'), 
    'TFJS_PATH':os.path.join('Tensorflow', 'workspace','models',CUSTOM_MODEL_NAME, 'tfjsexport'), 
    'TFLITE_PATH':os.path.join('Tensorflow', 'workspace','models',CUSTOM_MODEL_NAME, 'tfliteexport'), 
    'PROTOC_PATH':os.path.join('Tensorflow','protoc')
 }

In [5]:
files = {
    'PIPELINE_CONFIG':os.path.join(paths['CHECKPOINT_PATH'], name + '_pipeline.config'),
    'TF_RECORD_SCRIPT': os.path.join(paths['SCRIPTS_PATH'], TF_RECORD_SCRIPT_NAME), 
    'LABELMAP': os.path.join(paths['ANNOTATION_PATH'], LABEL_MAP_NAME)
}

In [6]:
TRAINING_SCRIPT = os.path.join(paths['APIMODEL_PATH'], 'research', 'object_detection', 'model_main_tf2.py')

# Подгружаем модель

In [15]:
if os.name =='posix':
    !wget {PRETRAINED_MODEL_URL}
    !mv {PRETRAINED_MODEL_NAME+'.tar.gz'} {paths['PRETRAINED_MODEL_PATH']}
    !cd {paths['PRETRAINED_MODEL_PATH']} && tar -zxvf {PRETRAINED_MODEL_NAME+'.tar.gz'}
if os.name == 'nt':
    wget.download(PRETRAINED_MODEL_URL)
    !move {PRETRAINED_MODEL_NAME+'.tar.gz'} {paths['PRETRAINED_MODEL_PATH']}
    !cd {paths['PRETRAINED_MODEL_PATH']} && tar -zxvf {PRETRAINED_MODEL_NAME+'.tar.gz'}

--2024-07-28 18:56:44--  http://download.tensorflow.org/models/object_detection/tf2/20200711/ssd_mobilenet_v2_fpnlite_320x320_coco17_tpu-8.tar.gz
Resolving download.tensorflow.org (download.tensorflow.org)... 142.250.74.27, 142.250.74.59, 142.250.74.91, ...
Connecting to download.tensorflow.org (download.tensorflow.org)|142.250.74.27|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 20515344 (20M) [application/x-tar]
Saving to: ‘ssd_mobilenet_v2_fpnlite_320x320_coco17_tpu-8.tar.gz’

ssd_mobilenet_v2_fp 100%[===================>]  19.56M  2.34MB/s    in 13s     

2024-07-28 18:57:00 (1.49 MB/s) - ‘ssd_mobilenet_v2_fpnlite_320x320_coco17_tpu-8.tar.gz’ saved [20515344/20515344]

ssd_mobilenet_v2_fpnlite_320x320_coco17_tpu-8/
ssd_mobilenet_v2_fpnlite_320x320_coco17_tpu-8/checkpoint/
ssd_mobilenet_v2_fpnlite_320x320_coco17_tpu-8/checkpoint/ckpt-0.data-00000-of-00001
ssd_mobilenet_v2_fpnlite_320x320_coco17_tpu-8/checkpoint/checkpoint
ssd_mobilenet_v2_fpnlite_320x320_co

Копируем конфиг в тренировочную папку

In [16]:
if os.name =='posix':
    !cp {os.path.join(paths['PRETRAINED_MODEL_PATH'], PRETRAINED_MODEL_NAME, 'pipeline.config')} {os.path.join(paths['CHECKPOINT_PATH'], name + '_pipeline.config')}
if os.name == 'nt':
    !copy {os.path.join(paths['PRETRAINED_MODEL_PATH'], PRETRAINED_MODEL_NAME, 'pipeline.config')} {os.path.join(paths['CHECKPOINT_PATH'], name + '_pipeline.config')}

In [17]:
os.path.join(paths['PRETRAINED_MODEL_PATH'], PRETRAINED_MODEL_NAME, 'pipeline.config')

'Tensorflow/workspace/pre-trained-models/ssd_mobilenet_v2_fpnlite_320x320_coco17_tpu-8/pipeline.config'

# Обновляем конфиг под наши данные

In [24]:
files['PIPELINE_CONFIG']

'Tensorflow/workspace/models/my_ssd_mobnet/osetr_checkpoints/osetr_pipeline.config'

In [21]:
files['LABELMAP']

'Tensorflow/workspace/annotations/label_map_osetr.pbtxt'

In [22]:
with open(files['LABELMAP']) as f:
    lines = f.readlines()
label_count = lines.count('}\n')

In [23]:
label_count

9

In [20]:
# pipeline_config = pipeline_pb2.TrainEvalPipelineConfig()
# with tf.io.gfile.GFile(files['PIPELINE_CONFIG'], "r") as f:                                                                                                                                                                                                                     
#     proto_str = f.read()                                                                                                                                                                                                                                          
#     text_format.Merge(proto_str, pipeline_config)  

In [29]:
# pipeline_config.model.ssd.num_classes = label_count
# pipeline_config.train_config.batch_size = 2
# pipeline_config.train_config.fine_tune_checkpoint = os.path.join(paths['PRETRAINED_MODEL_PATH'], PRETRAINED_MODEL_NAME, 'checkpoint', 'ckpt-0')
# pipeline_config.train_config.fine_tune_checkpoint_type = "detection"
# pipeline_config.train_input_reader.label_map_path= files['LABELMAP']
# pipeline_config.train_input_reader.tf_record_input_reader.input_path[:] = [os.path.join(paths['ANNOTATION_PATH'], name + '_train.record')]
# pipeline_config.eval_input_reader[0].label_map_path = files['LABELMAP']
# pipeline_config.eval_input_reader[0].tf_record_input_reader.input_path[:] = [os.path.join(paths['ANNOTATION_PATH'], name + '_test.record')]

In [30]:
# config_text = text_format.MessageToString(pipeline_config)                                                                                                                                                                                                        
# with tf.io.gfile.GFile(files['PIPELINE_CONFIG'], "wb") as f:                                                                                                                                                                                                                     
#     f.write(config_text)   

In [31]:
def update_pipeline_config(config_path, label_map_path, train_record_path, test_record_path, checkpoint_path):
    pipeline_config = pipeline_pb2.TrainEvalPipelineConfig()

    # Read the existing pipeline config
    with open(config_path, 'r') as f:
        proto_str = f.read()
        # proto_str = proto_str.replace("fine_tune_checkpoint_version: V2", "")
        text_format.Merge(proto_str, pipeline_config)


    # Update the paths
    pipeline_config.model.ssd.num_classes = label_count
    pipeline_config.train_config.batch_size = 3
    pipeline_config.train_config.fine_tune_checkpoint = os.path.join(paths['PRETRAINED_MODEL_PATH'], PRETRAINED_MODEL_NAME, 'checkpoint', 'ckpt-0')
    pipeline_config.train_config.fine_tune_checkpoint_type = "detection"
    pipeline_config.train_input_reader.label_map_path= files['LABELMAP']
    pipeline_config.train_input_reader.tf_record_input_reader.input_path[:] = [os.path.join(paths['ANNOTATION_PATH'], name + '_train.record')]
    pipeline_config.eval_input_reader[0].label_map_path = files['LABELMAP']
    pipeline_config.eval_input_reader[0].tf_record_input_reader.input_path[:] = [os.path.join(paths['ANNOTATION_PATH'], name + '_test.record')]

    # Write the updated pipeline config back to the file
    with open(config_path, 'w') as f:
        f.write(text_format.MessageToString(pipeline_config))

In [32]:
update_pipeline_config(
    config_path=files['PIPELINE_CONFIG'],
    label_map_path=files['LABELMAP'],
    train_record_path=os.path.join(paths['ANNOTATION_PATH'], name + '_train.record'),
    test_record_path=os.path.join(paths['ANNOTATION_PATH'], name + '_test.record'),
    checkpoint_path=paths['CHECKPOINT_PATH']
)

# Обучение модели

In [9]:
command = "python3 {} --model_dir={} --pipeline_config_path={} --num_train_steps=2000".format(TRAINING_SCRIPT, paths['CHECKPOINT_PATH'],files['PIPELINE_CONFIG'])

Запустить через терминал (внутри venv!)

In [10]:
print(command)

python3 Tensorflow/models/research/object_detection/model_main_tf2.py --model_dir=Tensorflow/workspace/models/my_ssd_mobnet/osetr_checkpoints --pipeline_config_path=Tensorflow/workspace/models/my_ssd_mobnet/osetr_checkpoints/osetr_pipeline.config --num_train_steps=2000


In [11]:
%run Tensorflow/models/research/object_detection/model_main_tf2.py --model_dir=Tensorflow/workspace/models/my_ssd_mobnet/osetr_checkpoints --pipeline_config_path=Tensorflow/workspace/models/my_ssd_mobnet/pipeline.config --num_train_steps=20000

INFO:tensorflow:Using MirroredStrategy with devices ('/job:localhost/replica:0/task:0/device:GPU:0',)


I0728 18:30:43.979378 123802359001088 mirrored_strategy.py:419] Using MirroredStrategy with devices ('/job:localhost/replica:0/task:0/device:GPU:0',)


INFO:tensorflow:Maybe overwriting train_steps: 20000


I0728 18:30:44.031571 123802359001088 config_util.py:552] Maybe overwriting train_steps: 20000


INFO:tensorflow:Maybe overwriting use_bfloat16: False


...

I0728 18:42:26.743675 123802359001088 model_lib_v2.py:708] {'Loss/classification_loss': 0.55906177,
 'Loss/localization_loss': 0.06558376,
 'Loss/regularization_loss': 0.06862474,
 'Loss/total_loss': 0.69327027,
 'learning_rate': 0.054054994}


INFO:tensorflow:Step 20000 per-step time 0.229s


I0728 18:42:49.700241 123802359001088 model_lib_v2.py:705] Step 20000 per-step time 0.229s


INFO:tensorflow:{'Loss/classification_loss': 0.5223027,
 'Loss/localization_loss': 0.011995207,
 'Loss/regularization_loss': 0.06840765,
 'Loss/total_loss': 0.60270554,
 'learning_rate': 0.0538146}


I0728 18:42:49.702064 123802359001088 model_lib_v2.py:708] {'Loss/classification_loss': 0.5223027,
 'Loss/localization_loss': 0.011995207,
 'Loss/regularization_loss': 0.06840765,
 'Loss/total_loss': 0.60270554,
 'learning_rate': 0.0538146}


Нужно опять поменять код внутри библиотеки
https://github.com/tensorflow/models/issues/11099

Внизу команда для терминала, которая позволяет посмотреть показатели обучения

# Оценивание модели

нихера не работает eval

In [7]:
os.environ['CUDA_VISIBLE_DEVICES'] = '-1'

In [8]:
command = "python3 {} --model_dir={} --pipeline_config_path={} --checkpoint_dir={}".format(TRAINING_SCRIPT, paths['CHECKPOINT_PATH'],files['PIPELINE_CONFIG'], paths['CHECKPOINT_PATH'])

Запустить через терминал (внутри venv!)

In [9]:
print(command)

python3 Tensorflow/models/research/object_detection/model_main_tf2.py --model_dir=Tensorflow/workspace/models/my_ssd_mobnet/osetr_checkpoints --pipeline_config_path=Tensorflow/workspace/models/my_ssd_mobnet/osetr_checkpoints/osetr_pipeline.config --checkpoint_dir=Tensorflow/workspace/models/my_ssd_mobnet/osetr_checkpoints


In [10]:
%run Tensorflow/models/research/object_detection/model_main_tf2.py --model_dir=Tensorflow/workspace/models/my_ssd_mobnet/osetr_checkpoints/ --pipeline_config_path=Tensorflow/workspace/models/my_ssd_mobnet/osetr_checkpoints/osetr_pipeline.config --checkpoint_dir=Tensorflow/workspace/models/my_ssd_mobnet/osetr_checkpoints/

2024-07-28 21:18:15.361562: E tensorflow/compiler/xla/stream_executor/cuda/cuda_driver.cc:268] failed call to cuInit: CUDA_ERROR_NO_DEVICE: no CUDA-capable device is detected
W0728 21:18:15.364933 137997177622528 model_lib_v2.py:1089] Forced number of epochs for all eval validations to be 1.


INFO:tensorflow:Maybe overwriting sample_1_of_n_eval_examples: None


I0728 21:18:15.365810 137997177622528 config_util.py:552] Maybe overwriting sample_1_of_n_eval_examples: None


INFO:tensorflow:Maybe overwriting use_bfloat16: False


I0728 21:18:15.366576 137997177622528 config_util.py:552] Maybe overwriting use_bfloat16: False


...

INFO:tensorflow:Encountered Graph execution error:

Detected at node 'WeightSharedConvolutionalBoxPredictor/WeightSharedConvolutionalClassHead/Reshape_9' defined at (most recent call last):
    File "/usr/lib/python3.10/runpy.py", line 196, in _run_module_as_main
      return _run_code(code, main_globals, None,
    File "/usr/lib/python3.10/runpy.py", line 86, in _run_code
      exec(code, run_globals)
    File "/home/midnightknight/Blood-object-detection/.obj_det/lib/python3.10/site-packages/ipykernel_launcher.py", line 18, in <module>
      app.launch_new_instance()
    File "/home/midnightknight/Blood-object-detection/.obj_det/lib/python3.10/site-packages/traitlets/config/application.py", line 1075, in launch_instance
      app.start()
    File "/home/midnightknight/Blood-object-detection/.obj_det/lib/python3.10/site-packages/ipykernel/kernelapp.py", line 739, in start
      self.io_loop.start()
    File "/home/midnightknight/Blood-object-detection/.obj_det/lib/python3.10/site-packa

I0728 21:18:34.844198 137997177622528 model_lib_v2.py:942] Encountered Graph execution error:

Detected at node 'WeightSharedConvolutionalBoxPredictor/WeightSharedConvolutionalClassHead/Reshape_9' defined at (most recent call last):
    File "/usr/lib/python3.10/runpy.py", line 196, in _run_module_as_main
      return _run_code(code, main_globals, None,
    File "/usr/lib/python3.10/runpy.py", line 86, in _run_code
      exec(code, run_globals)
    File "/home/midnightknight/Blood-object-detection/.obj_det/lib/python3.10/site-packages/ipykernel_launcher.py", line 18, in <module>
      app.launch_new_instance()
    File "/home/midnightknight/Blood-object-detection/.obj_det/lib/python3.10/site-packages/traitlets/config/application.py", line 1075, in launch_instance
      app.start()
    File "/home/midnightknight/Blood-object-detection/.obj_det/lib/python3.10/site-packages/ipykernel/kernelapp.py", line 739, in start
      self.io_loop.start()
    File "/home/midnightknight/Blood-object-d

INFO:tensorflow:A replica probably exhausted all examples. Skipping pending examples on other replicas.


I0728 21:18:34.845594 137997177622528 model_lib_v2.py:943] A replica probably exhausted all examples. Skipping pending examples on other replicas.


TypeError: 'NoneType' object is not iterable